In [4]:
import pandas as pd
import os



In [6]:
os.getcwd()

'C:\\Users\\HP\\anaconda3'

In [13]:
os.chdir('C:\\Users\\HP\\Downloads\\enron1')

In [35]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

# Create DataFrames from records
df_ham = pd.DataFrame.from_records(ham)
df_spam = pd.DataFrame.from_records(spam)

# Concatenate DataFrames
df = pd.concat([df_ham, df_spam], ignore_index=True)

# Print or use the resulting DataFrame df as needed
print(df)


skipped 2248.2004-09-23.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt
                                name  \
0     0001.1999-12-10.farmer.ham.txt   
1     0002.1999-12-13.farmer.ham.txt   
2     0003.1999-12-14.farmer.ham.txt   
3     0004.1999-12-14.farmer.ham.txt   
4     0005.1999-12-14.farmer.ham.txt   
...                              ...   
5163     5163.2005-09-06.GP.spam.txt   
5164     5164.2005-09-06.GP.spam.txt   
5165     5167.2005-09-06.GP.spam.txt   
5166     5170.2005-09-06.GP.spam.txt   
5167     5171.2005-09-06.GP.spam.txt   

                                                content category  
0               Subject: christmas tree farm pictures\n      ham  
1     Subject: vastar resources , inc .\ngary , prod...      ham  
2     Subject: calpine daily gas nomination\n- calpi...      ham  
3     Subject: re : issue\nfyi - see note below - al...      ham  
4     Subject: meter 7268 nov allocation

In [36]:
import re

def preprocessor(e):
    return re.sub('[^A-Za-z]', ' ', e).lower()

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.
vectorizer = CountVectorizer(preprocessor=preprocessor)

# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.
X_train,X_test,y_train,y_test = train_test_split(df["content"],df["category"],test_size=0.2,random_state=1)

# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.
X_train_df = vectorizer.fit_transform(X_train)

# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.
model = LogisticRegression()
model.fit(X_train_df,y_train)

# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.
X_test_df = vectorizer.transform(X_test)
y_pred = model.predict(X_test_df)

# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
print(f'Accuracy:\n{accuracy_score(y_test,y_pred)}\n')
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}\n')
print(f'Detailed Statistics:\n{classification_report(y_test,y_pred)}\n')

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:
0.9748549323017408

Confusion Matrix:
[[729  16]
 [ 10 279]]

Detailed Statistics:
              precision    recall  f1-score   support

         ham       0.99      0.98      0.98       745
        spam       0.95      0.97      0.96       289

    accuracy                           0.97      1034
   macro avg       0.97      0.97      0.97      1034
weighted avg       0.98      0.97      0.97      1034




In [38]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.
features = vectorizer.get_feature_names_out()

# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
importance = model.coef_[0]

# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.
l = list(enumerate(importance))
print()
l.sort(key=lambda e: e[1], reverse=True)
for i,imp in l[:10]:
    print(imp, features[i])
print()
l.sort(key=lambda e: -e[1], reverse=True)
for i,imp in l[:10]:
    print(imp, features[i])


0.9465583107441584 http
0.8661511807584313 prices
0.8151801052435645 no
0.7514073777428096 pain
0.7224498270475626 only
0.7189657479854644 paliourg
0.7186422736056727 money
0.6930449003845621 more
0.6866426430710832 remove
0.6802618705871066 removed

-1.5991102521196645 enron
-1.5480736796575165 attached
-1.4054637695531058 daren
-1.382226358686004 thanks
-1.3566541123934064 doc
-1.1859262189523359 deal
-1.1422474341911573 xls
-1.124174639477865 meter
-1.060026121784396 hpl
-1.0217018113613179 neon


In [ ]:
ALL DONE!
